# Week 17
## Ensemble Learning and Random Forest

In [1]:
# Dependencies and modules:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours

from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline

from sklearn.metrics import classification_report, roc_curve, roc_auc_score

import matplotlib.pyplot as plt

from sklearn.svm import SVC

# diabetes.csv file:

# diabetes_path = "C:/Users/Nik/Documents/diabetes.csv"
# diabetes_df = pd.read_csv(diabetes_path)
# diabetes_df.head(1)

1. What is inductive reasoning? Deductive reasoning? Give an example of each, different from the examples given in class.

Using ONE of the following sources, complete the questions for only that source. 

Credit approval: https://archive.ics.uci.edu/ml/datasets/Statlog+%28Australian+Credit+Approval%29

Cardiac Arrhythmia: https://archive.ics.uci.edu/ml/datasets/Arrhythmia 

Abalone age: https://archive.ics.uci.edu/ml/datasets/Abalone - this one is a bit harder since it’s not binary like the others, but if you really want to master these concepts, you should pick this one. Use RMSE as a performance metric if you do this as regression. You should target a value of under 3.

Note: at least one of your models should have the most relevant performance metric above .90 . All performance metrics should be above .75 . You will partially be graded on model performance.


2. Preprocess your dataset. Indicate which steps worked and which didn’t. Include your thoughts on why certain steps worked and certain steps didn’t. 

3. Create a decision tree model tuned to the best of your abilities. Explain how you tuned it.

4. Create a random forest model tuned to the best of your abilities. Explain how you tuned it.

5. Create an xgboost model tuned to the best of your abilities. Explain how you tuned it. 

7. Which model performed best? What is your performance metric? Why? 

# DataCamp Completions: